## <center>Synchronous Computing: Heat Exchange</center>
### <center> Linh B. Ngo </center>
### <center> CPSC 4770/6770 </center>

<center> <img src="pictures/08/heatexchange1.png" width="700"/> 
</center>

<center> <img src="pictures/08/heatexchange2.png" width="700"/> 
</center>

<center> <img src="pictures/08/heatexchange3.png" width="700"/> 
</center>

<center> <img src="pictures/08/heatexchange4.png" width="700"/> 
</center>

<center> <img src="pictures/08/heatexchange5.png" width="700"/> 
</center>

#### <center> Row Exchange

- All processes interact with adjacent neighbors to exchange *ghost row* (*halo row*)
- All processes are to be synchronized such that the next iteration of the computation only happens after all row exchanges are completed

In [1]:
%%writefile codes/mpi4py/row_exchange.py
#!/usr/bin/env python
# row_exchange.py
import numpy as np
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank(); size = comm.Get_size(); N = 24
status = MPI.Status()

local_array = np.full(shape=(int(N / size) + 2, N), fill_value=rank, dtype="int")
full_array = np.zeros(shape=(N + size * 2, N), dtype="int")

comm.Gather(local_array, full_array, root=0)
if rank == 0:
    print (full_array)
if (rank < size - 1):
    print (rank)
    comm.Send(local_array[int(N/size),], dest = rank + 1, tag = 0)
    comm.Recv(local_array[int(N/size) + 1,], source = rank + 1, tag = 0, status = status)
if (rank > 0):
    print (rank)
    comm.Recv(local_array[0,], source = rank - 1, tag = 0, status = status)
    comm.Send(local_array[1,], dest = rank - 1, tag = 0)

comm.Barrier()
comm.Gather(local_array, full_array, root=0)
if rank == 0:
    print (full_array)

Overwriting codes/mpi4py/row_exchange.py


In [2]:
!chmod 755 codes/mpi4py/row_exchange.py
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 4 --mca mpi_cuda_support 0 codes/mpi4py/row_exchange.py

1
1
2
2
3
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2

#### <center> The Game of Life </center>

- Board game: theoretically infinite two-dimensional array of cells
- Each cell hold one "organism" and has eight neighboring cells, including those diagonally adjacent
- Initially, some cells occupied

** Rules: **
- two or three neighboring organisms: survives for the next generation
- more than four neighbors: dies from overpopulation
- less than two neighbors: dies from isolation
- Empty cell with exactly three neighbors: spawn new organism

** Serious applications: **
- fluid/gas dynamics
- movement of fluids and gases aroudn objects
- diffusion of gases
- biological growth
- airflow across an airplane wing
- erosion/movement of sand at a beach or riverbank